In [1]:
import pandas as pd
import env
import os
from sklearn.model_selection import train_test_split

# Planning

    - Find drivers for customer churn at Telco. Why are customers churning?

    - Construct a ML classification model that accurately predicts customer churn

    - Present your process and findings to the lead data scientist

# Acquisition
    - Data is acuired from MySQL 
    - its contains 7043 rows and 24 columns 

In [2]:
def get_telco_data(file_name="telco_churn.csv") -> pd.DataFrame:
    if os.path.isfile(file_name):
        return pd.read_csv(file_name)
    query = """SELECT * 
               FROM customers
               LEFT JOIN contract_types
               USING(contract_type_id)
               LEFT JOIN internet_service_types
               USING (internet_service_type_id)
               LEFT JOIN payment_types
               USING (payment_type_id)"""
    connection = get_connection("telco_churn")
    df = pd.read_sql(query, connection)
    df.to_csv(file_name, index=False)
    return df

In [3]:
df = get_telco_data()
df.shape

(7043, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [5]:
#use crosstab to check values
pd.crosstab(df.payment_type, df.payment_type_id)

payment_type_id,1,2,3,4
payment_type,,,,
Bank transfer (automatic),0,0,1544,0
Credit card (automatic),0,0,0,1522
Electronic check,2365,0,0,0
Mailed check,0,1612,0,0


In [6]:
#use crosstab to check values
pd.crosstab(df.contract_type, df.contract_type_id)

contract_type_id,1,2,3
contract_type,,,
Month-to-month,3875,0,0
One year,0,1473,0
Two year,0,0,1695


In [7]:
#use crosstab to check values
pd.crosstab(df.internet_service_type, df.internet_service_type_id)

internet_service_type_id,1,2
internet_service_type,,
DSL,2421,0
Fiber optic,0,3096


## Can drop the duplicate columns ^  

In [8]:
df.isnull().sum() # check for null values 

payment_type_id                0
internet_service_type_id       0
contract_type_id               0
customer_id                    0
gender                         0
senior_citizen                 0
partner                        0
dependents                     0
tenure                         0
phone_service                  0
multiple_lines                 0
online_security                0
online_backup                  0
device_protection              0
tech_support                   0
streaming_tv                   0
streaming_movies               0
paperless_billing              0
monthly_charges                0
total_charges                  0
churn                          0
contract_type                  0
internet_service_type       1526
payment_type                   0
dtype: int64

In [9]:
df.internet_service_type.value_counts() # identify why its null

internet_service_type
Fiber optic    3096
DSL            2421
Name: count, dtype: int64

In [10]:
df['internet_service_type'] = df['internet_service_type'].fillna('No internet service') # eval if I drop or fill null values
df.internet_service_type.value_counts()

internet_service_type
Fiber optic            3096
DSL                    2421
No internet service    1526
Name: count, dtype: int64

In [11]:
df.isnull().sum() # verify 

payment_type_id             0
internet_service_type_id    0
contract_type_id            0
customer_id                 0
gender                      0
senior_citizen              0
partner                     0
dependents                  0
tenure                      0
phone_service               0
multiple_lines              0
online_security             0
online_backup               0
device_protection           0
tech_support                0
streaming_tv                0
streaming_movies            0
paperless_billing           0
monthly_charges             0
total_charges               0
churn                       0
contract_type               0
internet_service_type       0
payment_type                0
dtype: int64

## filled null values ^  

In [12]:
df.head().T # identify columns to be encoded

,0,1,2,3,4
payment_type_id,2,2,1,1,2
internet_service_type_id,1,1,2,2,2
contract_type_id,2,1,1,1,1
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ
gender,Female,Male,Male,Male,Female
senior_citizen,0,0,0,1,1
partner,Yes,No,No,Yes,Yes
dependents,Yes,No,No,No,No
tenure,9,9,4,13,3
phone_service,Yes,Yes,Yes,Yes,Yes


In [13]:
# encode all binary categorical variables

df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})

# encoded all binary categorical variables into numeric values ^ 

In [15]:
dummy_df = pd.get_dummies(df[['multiple_lines',
                                     'online_security',
                                     'online_backup',
                                     'device_protection',
                                     'tech_support',
                                     'streaming_tv',
                                     'streaming_movies',
                                     'contract_type',
                                     'internet_service_type',
                                     'payment_type']],
                                  drop_first=True).astype(int)

dummy_df.head().T # create new df to hold dummy variables

,0,1,2,3,4
multiple_lines_No phone service,0,0,0,0,0
multiple_lines_Yes,0,1,0,0,0
online_security_No internet service,0,0,0,0,0
online_security_Yes,0,0,0,0,0
online_backup_No internet service,0,0,0,0,0
online_backup_Yes,1,0,0,1,0
device_protection_No internet service,0,0,0,0,0
device_protection_Yes,0,0,1,1,0
tech_support_No internet service,0,0,0,0,0
tech_support_Yes,1,0,0,0,1


In [16]:
df = pd.concat( [df, dummy_df], axis=1 ) # put both df together


## encoded the rest of the categorical varables ^ 

In [19]:
df.head().T # verify 

,0,1,2,3,4
payment_type_id,2,2,1,1,2
internet_service_type_id,1,1,2,2,2
contract_type_id,2,1,1,1,1
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ
gender,Female,Male,Male,Male,Female
senior_citizen,0,0,0,1,1
partner,Yes,No,No,Yes,Yes
dependents,Yes,No,No,No,No
tenure,9,9,4,13,3
phone_service,Yes,Yes,Yes,Yes,Yes


In [20]:
df.total_charges.value_counts() # check formatiing

total_charges
           11
20.2       11
19.75       9
19.9        8
20.05       8
           ..
2387.75     1
6302.8      1
2058.5      1
829.55      1
3707.6      1
Name: count, Length: 6531, dtype: int64

In [21]:
df.total_charges = df.total_charges.str.replace(' ', '0').astype(float) # remove empty space and fill it with 0


In [22]:
df.total_charges.value_counts() # verity 

total_charges
0.00       11
20.20      11
19.75       9
19.90       8
20.05       8
           ..
2387.75     1
6302.80     1
2058.50     1
829.55      1
3707.60     1
Name: count, Length: 6531, dtype: int64

## fit formating ^ 

In [23]:
def prep_telco(df):
    df = df.drop(columns=['internet_service_type_id', 'contract_type_id', 'payment_type_id'])
    df['internet_service_type'] = df['internet_service_type'].fillna('No internet service')


    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})

    dummy_df = pd.get_dummies(df[['multiple_lines',
                                     'online_security',
                                     'online_backup',
                                     'device_protection',
                                     'tech_support',
                                     'streaming_tv',
                                     'streaming_movies',
                                     'contract_type',
                                     'internet_service_type',
                                     'payment_type']],
                                  drop_first=True).astype(int)

    df = pd.concat( [df, dummy_df], axis=1 )

    df.total_charges = df.total_charges.str.replace(' ', '0').astype(float)

    return df

## create a function ^ 